# 👋🌍 Hello, world: Submit a Cirq job to IonQ

In this notebook, we'll review the basics of Azure Quantum by submitting a simple *job*, or quantum program, to [IonQ](https://ionq.com/). We will use [Cirq](https://quantumai.google/cirq) to express the quantum job.

## Submit a simple job to IonQ using Azure Quantum
Azure Quantum provides several ways to express quantum programs. In this example we are using Cirq, but note that Q# and Qiskit are also supported. All code in this example will be written in Python.

Let's begin. When you see a code block, hover over it and click the triangle play-button to execute it. To avoid any compilation issues, this should be done in order from top to bottom.

### 1. Connect to the Azure Quantum workspace

To connect to the Azure Quantum service, construct an instance of the `AzureQuantumService`. Note that it's imported from `azure.quantum.cirq`.

In [ ]:
from azure.quantum.cirq import AzureQuantumService

service = AzureQuantumService(
    resource_id = "",
    location = ""
)

Let's see what providers and targets are enabled in this workspace with the following command:


In [ ]:
print("This workspace's targets:")
for target in service.targets():
    print("-", target.name)

### ❕ Do you see `ionq.simulator` in your list of targets? If so, you're ready to keep going.

Don't see it? You may need to add IonQ to your workspace to run this sample. Navigate to the **Providers** page in the portal and click **+Add** to add the IonQ provider. Don't worry, there's a free credits plan available.

## IonQ: The quantum provider
Azure Quantum partners with third-party companies to deliver solutions to quantum jobs. These company offerings are called *providers*. Each provider can offer multiple *targets* with different capabilities. See the table below for IonQ's targets.

| Target name | Target ID | Number of qubits | Description |
| --- | --- | --- | --- |
| Quantum simulator | `ionq.simulator` | 29 qubits | IonQ's cloud-based idealized simulator. Free of cost. |
| Aria 1 | `ionq.qpu.aria-1` | 23 qubits | IonQ's Aria 1 trapped-ion quantum computer. This is real quantum hardware, not a simulation. |
| Quantum computer | `ionq.qpu` | 11 qubits | IonQ's trapped-ion quantum computer. This is real quantum hardware, not a simulation. |

For this example, we will use `ionq.simulator`. To learn more about IonQ's targets, check out our [documentation](https://docs.microsoft.com/azure/quantum/provider-ionq).

### 2. Build the quantum program

Let's create a simple Cirq circuit to run.

In [ ]:
import cirq

q0 = cirq.LineQubit(0)
circuit = cirq.Circuit(
    cirq.H(q0),               # Apply an H-gate to q0
    cirq.measure(q0)          # Measure q0
)
circuit

The circuit you built is a simple quantum random bit generator. With IonQ's idealized simulator, we will be able to calculate the probability of measuring a `1` or `0`.

### 3. Submit the quantum program to IonQ

In [ ]:
# Using the IonQ simulator target, call "run" to submit the job. We'll
# use 100 repetitions (simulated runs).
job = service.targets("ionq.simulator").submit(circuit, name="hello world-cirq-ionq", repetitions=100)

# Print the job ID.
print("Job id:", job.job_id())

The job ID can be used to retrieve the results later using the [get_details function](https://docs.microsoft.com/azure/quantum/optimization-job-reference#jobdetails) or by viewing it under the **Job management** section of the portal.

### 4. Obtain the job results
Let's await the job execution by calling `job.results()`. This may take a minute or so ⏳. Your job is being packaged and sent to IonQ, where it will wait its turn to be run.

In [ ]:
# Await job results.
print("Awaiting job results...")
result = job.results()

# To view the probabilities computed for each Qubit state, you can print the result.
print("Job finished. State probabilities:")
print(result)

You can also view a histogram of the results which computes hypothetical states based on the result's computed probabilities. Try running the next code cell multiple times to see the value change in accordance with the probabilities that the simulation computed.

In [ ]:
from cirq.vis import plot_state_histogram

# Convert the result to a cirq result by "measuring" the returned
# qubits a number of times equal to the specified number of repetitions.
measurement = result.to_cirq_result()

# We can now use Cirq to plot the results of the "measurement."
res = plot_state_histogram(measurement)

**See the histogram above? Congratulations, you've submitted a job with Azure Quantum! 👏**


### 5. Estimate costs

To estimate the costs of running this program on a simulator or hardware, you can use the `service.estimate_cost` method.

In [ ]:
cost = service.estimate_cost(circuit, repetitions=100, target="ionq.qpu")
print(f"Estimated cost: {cost.estimated_total} {cost.currency_code}")

### 6. Next steps
Next, you can try running a program on IonQ's hardware target. Just replace `ionq.simulator` with `ionq.qpu`. Or try another sample by navigating back to the sample gallery. The same "hello world" sample can be run with different quantum providers by choosing another option in the gallery card drop-down menu. Don't worry - your work here is automatically saved.

To learn more about submitting Cirq circuits to Azure Quantum, review [this documentation](https://docs.microsoft.com/azure/quantum/quickstart-microsoft-cirq?pivots=platform-ionq).

To learn more about job pricing, review the [Azure Quantum documentation on job costs](https://docs.microsoft.com/azure/quantum/azure-quantum-job-costs).